In [46]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from ckks_encoder import CKKSEncoder
from ckks import Params,CKKS
from plaintext import Plaintext
from poly import Polynomial
import math 
from math import floor


In [47]:

params = Params(8,  1<<600,1<<1200)
ckks= CKKS(params)
public_key,private_key=ckks.generate_keys()


Initialised


In [48]:
def read_dataset(url,col):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [49]:
X_train, X_test, y_train, y_test = read_dataset('./dataset/data.csv',1)

(143, 30)


C:\Users\delwy\AppData\Local\Temp\ipykernel_9700\1684536102.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
C:\Users\delwy\AppData\Local\Temp\ipykernel_9700\1684536102.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


In [50]:
X_train

array([[-0.50754064, -1.5873111 , -0.53291875, ..., -0.29626036,
        -0.09794363, -0.92683077],
       [-0.9010532 , -1.5964096 , -0.92588454, ..., -1.2394603 ,
        -1.1854004 , -1.0006467 ],
       [-1.1103684 ,  0.05040994, -1.1043581 , ..., -1.3858792 ,
        -0.9962085 , -0.9435547 ],
       ...,
       [-0.9317526 , -2.1969075 , -0.9408586 , ..., -1.0250068 ,
        -1.2696624 , -0.3916628 ],
       [-0.73639184, -0.20889577, -0.75833786, ..., -0.72958136,
        -0.23626071, -1.0906105 ],
       [-0.37916064, -0.65472007, -0.43093362, ..., -0.83033943,
        -0.80383676, -1.1350157 ]], dtype=float32)

In [51]:
def custom_relu(x):
    return x +x*x
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(2),
    # tf.keras.layers.Activation(custom_relu),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/30
27/27 [==============================] - 1s 2ms/step - loss: 0.4661 - accuracy: 0.8169
Epoch 2/30
27/27 [==============================] - 0s 2ms/step - loss: 0.3381 - accuracy: 0.8803
Epoch 3/30
27/27 [==============================] - 0s 2ms/step - loss: 0.2726 - accuracy: 0.9155
Epoch 4/30
27/27 [==============================] - 0s 2ms/step - loss: 0.2338 - accuracy: 0.9202
Epoch 5/30
27/27 [==============================] - 0s 4ms/step - loss: 0.2088 - accuracy: 0.9249
Epoch 6/30
27/27 [==============================] - 0s 3ms/step - loss: 0.1899 - accuracy: 0.9296
Epoch 7/30
27/27 [==============================] - 0s 2ms/step - loss: 0.1754 - accuracy: 0.9343
Epoch 8/30
27/27 [==============================] - 0s 2ms/step - loss: 0.1641 - accuracy: 0.9343
Epoch 9/30
27/27 [==============================] - 0s 2ms/step - loss: 0.1542 - accuracy: 0.9366
Epoch 10/30
27/27 [==============================] - 0s 2ms/step - loss: 0.1462 - accuracy: 0.9413
Epoch 11/30
27/27 [

In [52]:
def create_model_weights_file(model):
  i=0
  for layer in model.layers:
    if('activation' in layer.name):
      continue
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [53]:
create_model_weights_file(model)

In [54]:
with open('layer0.npy', 'rb') as f:
              print(np.load(f))
              print(np.load(f))

[[-5.73743045e-01 -1.78259104e-01  1.38971005e-02 -2.50905782e-01
  -6.43843189e-02 -3.23205978e-01  7.56815150e-02 -5.14242053e-01
  -7.87992105e-02  4.34094965e-01 -3.59391868e-01  1.26313165e-01
  -1.41525939e-01 -4.37217832e-01  2.79651940e-01  9.45818238e-03
   2.58637786e-01 -4.39461581e-02 -1.23167038e-01 -8.37039798e-02
  -6.61692679e-01 -1.38493299e-01 -6.31045520e-01 -2.67279238e-01
   1.89797338e-02 -4.52327847e-01 -6.61443248e-02 -2.85920531e-01
  -1.02842726e-01  8.41433927e-03]
 [-5.59416950e-01 -1.31857485e-01  1.39319487e-02 -1.38063386e-01
  -9.09526572e-02 -3.19924206e-02 -8.62172991e-02 -2.20765799e-01
  -1.09611556e-01 -1.63946584e-01  1.26403674e-01 -1.41077161e-01
   6.73070550e-04 -6.34251475e-01 -3.45513999e-01  2.34230548e-01
   3.33973169e-01 -2.57029474e-01  2.21261010e-02  1.96516946e-01
   2.71897838e-02 -6.91346109e-01  1.57559186e-01 -4.03240979e-01
  -1.49614319e-01 -3.94246638e-01 -2.84126312e-01  7.35369101e-02
   1.74779315e-02  1.01764193e-02]]
[0.14

In [59]:
def encode_value(a,scale):
    encoder=CKKSEncoder(16,scale)
    a_encoded=encoder.encode(np.array([a]*4))
    a_encoded=Polynomial(a_encoded.coef)
    a_encoded.poly_floor()
    a_pt=Plaintext(a_encoded,scale)
    return a_pt
def encrypt_input(x,scale):
    res=[]
    for item in x:
        item_pt=encode_value(item,scale)
        item_encrypted=public_key.encrypt(item_pt)
        res.append(item_encrypted)
    return res
    
    

In [60]:
class Model:
    def __init__(self,n_features,n_layers,n_layer_neurons,layer_activations,layer_weight_files,encrypted):
        self.n_features=n_features
        self.n_layers=n_layers
        self.n_layer_neurons=n_layer_neurons
        self.layer_activations=layer_activations
        self.weights=[]
        self.biases=[]
        self.encrypted=encrypted
        layer_number=0
        for wt_file in layer_weight_files:
            with open(wt_file, 'rb') as f:
              if self.encrypted==True:
                  w=np.load(f)
                  r,c=np.shape(w)
                  res_w=[]
                  for i in range(r):
                      row=[]
                      for j in range(c):
                          row.append(encode_value(w[i][j],1<<20))
                      res_w.append(row)
                  b=np.load(f)
                  res_b=[]
                  for i in b:
                      res_b.append(encode_value(i,(1<<20)**(layer_number+2)))
                  self.weights.append(res_w)
                  self.biases.append(res_b)
                  layer_number+=1
                   
              
              else:
                self.weights.append(np.load(f).tolist())
                self.biases.append(np.load(f).tolist())
                
    def __matmul(self,a,b):
        ## Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list 
        output=[]
        for i in range(len(a)):
            res=a[i][0]*b[0]
            for j in range(1,len(b)):
                res=a[i][j]*b[j]+res
            output.append(res)
        return output
    
    def __matadd(self,a,b):
        ## Input: a and b to be 1-D list
        # Output : a 1-D list 
        ## For memory saving b is modified in calculations so be careful about order of inputs
        
        for i in range(len(a)):
            b[i]=b[i]+a[i]
        return b  
    
    def infer(self,x):
        for i in range(self.n_layers):
            
            new_x=self.__matmul(self.weights[i],x)
            new_x=self.__matadd(self.biases[i],new_x)
            
            if(self.layer_activations[i]=='relu'):
               
                x=[t**2+t for t in new_x]
                # if(encrypted==True):
                #     temp=copy.deepcopy(new_x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                #     temp=copy.deepcopy(x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                if(self.encrypted==True):
                    x=[t**2+t for t in new_x]
                # else:
                #     x=[max(0,t) for t in new_x]
            else:  
                x=new_x  
        return x[0]


In [63]:
model_encrypted=Model(30,2,[2,1],['none','none'],['layer0.npy','layer1.npy'],True)
model_unencrypted=Model(30,2,[2,1],['none','none'],['layer0.npy','layer1.npy'],False)

In [64]:
normal_acc=0
encrypted_acc=0
limit=len(X_test)
for i in range(limit):
    x=X_test[i].tolist()
    x_encrypted=encrypt_input(x,1<<20)
    normal_res=model_unencrypted.infer(x)
    pt_dash=private_key.decrypt(model_encrypted.infer(x_encrypted))
    pt=np.polynomial.Polynomial(pt_dash.poly.convert_to_list())
    encoder=CKKSEncoder(16,pt_dash.scale)
    pt=encoder.decode(pt)
    encrypted_res=pt[0].real
    #print('Unencryted Inference:'+str(normal_res)+" Encrypted Inference:"+str(encrypted_res))
    if (normal_res>0 and y_test[i]==1) or (normal_res<=0 and y_test[i]==0):
        normal_acc+=1
    if (encrypted_res>0 and y_test[i]==1) or (encrypted_res<=0 and y_test[i]==0):
        encrypted_acc+=1
print('Accuracy on unencrypted Test Data:'+str(normal_acc/limit))
print('Accuracy on encrypted test data:'+str(encrypted_acc/limit))
  

Accuracy on unencrypted Test Data:0.958041958041958
Accuracy on encrypted test data:0.9440559440559441
